## команды запуска среды

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app-1")

In [4]:
from pyspark.sql import SparkSession, Row

In [5]:
import sys, os

##spark = ## старт вашей спарк сессии на машине
spark = SparkSession(sc)

## материал семинара

In [6]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window ## пр-во методов под оконные ф-ии, но мы в семинаре их не рассмортим

In [7]:
## учебный пример фрема данных на семинар

In [8]:
spark ## смотрим на описание запущенной мной спарк сессии

In [9]:
## здесь создаем тестовый пример набора данных, с которым будем работать в рамках семинара

sc = spark.sparkContext

test_data = [
{"name":"Moscow", "country":"Russia", "continent": "Europe", "population": 100_000_000},
{ "name":"Madrid", "country":"Spain" },
{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 205_000_000},
{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 140_000_008},
{ "name":"Barselona", "country":"Spain", "continent": "Europe" },
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 45_000_001 },
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 45_000_001 },
]

## создаем rdd
rdd = sc.parallelize(test_data)

## создаем DataFrame абстракцию поверх rdd
df = spark.read.json(rdd).localCheckpoint()

df

DataFrame[continent: string, country: string, name: string, population: bigint]

In [9]:
!pwd ## команда для получения пути до вашей home папки и места запуска ноутбука

/home/ubuntu


In [29]:
cat population.csv | head -n 10

continent,country,name,population
Europe,Russia,Moscow,100000000.0
,Spain,Madrid,
Europe,France,Paris,205000000.0
Europe,Germany,Berlin,140000008.0
Europe,Spain,Barselona,
Africa,Egypt,Cairo,45000001.0
Africa,Egypt,Cairo,45000001.0


In [36]:
!hdfs dfs -put population.csv /user/ubuntu/population.csv

In [37]:
!hdfs dfs -ls -h /user/ubuntu/ ## здесь смотрим на путь до моей юзер папки на HDFS!!!

Found 2 items
drwxr-xr-x   - ubuntu hadoop          0 2026-02-16 12:18 /user/ubuntu/.sparkStaging
-rw-r--r--   1 ubuntu hadoop        232 2026-02-16 12:21 /user/ubuntu/population.csv


In [38]:
# чтение файла по пути папки под юзером на HDFS

read_df = (
    spark
    .read
    .format("csv")
    .options(
        header=True,
        inferSchema=True,
        sep=","
    )
    .load("/user/ubuntu/population.csv")

)

read_df # <-  сформированный DataFrame

DataFrame[continent: string, country: string, name: string, population: double]

In [39]:
read_df.show(vertical=True) # вызываем action, смотрим данные в вертикальном разрезе

-RECORD 0------------------
 continent  | Europe       
 country    | Russia       
 name       | Moscow       
 population | 1.0E8        
-RECORD 1------------------
 continent  | null         
 country    | Spain        
 name       | Madrid       
 population | null         
-RECORD 2------------------
 continent  | Europe       
 country    | France       
 name       | Paris        
 population | 2.05E8       
-RECORD 3------------------
 continent  | Europe       
 country    | Germany      
 name       | Berlin       
 population | 1.40000008E8 
-RECORD 4------------------
 continent  | Europe       
 country    | Spain        
 name       | Barselona    
 population | null         
-RECORD 5------------------
 continent  | Africa       
 country    | Egypt        
 name       | Cairo        
 population | 4.5000001E7  
-RECORD 6------------------
 continent  | Africa       
 country    | Egypt        
 name       | Cairo        
 population | 4.5000001E7  



In [40]:
#запись файла по пути до папки юзера на HDFS

(
    read_df
    .write
    .format("parquet")
    .mode("overwrite")
    .save("/user/ubuntu/population_parquet")
)

In [41]:
!hdfs dfs -ls -h /user/ubuntu/population_parquet/ # тут наблюдаем за тем как файл успешно записался

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2026-02-16 12:21 /user/ubuntu/population_parquet/_SUCCESS
-rw-r--r--   1 ubuntu hadoop      1.3 K 2026-02-16 12:21 /user/ubuntu/population_parquet/part-00000-2b67caa5-f14f-4309-919a-0a53a2ff3494-c000.snappy.parquet


In [42]:
!hdfs dfs -cat /user/ubuntu/population_parquet/* # тут попытались вывести файл на чтение в консоль
# но формат паркет хранит данные бинарно, так что просто так их не прочитать

PAR1(,L  L   Europe   Africa ,6(EuropeAfrica   	    }0bdL
  1H   Russia   SpaindFrance   Germany   Egypt ,6 (SpainEgypt   (   �zxL  =$   Moscow
4adrid   ParislBerlin	   Barselona   Cairo ,6 (Paris	Barselona   (   ��@BL   8    �חA   �p�0v��A   *u�A ,   �p�A   *u�A(   �p�A   *u�A   
$   m�\Hspark_schema %	continent%  %country%  %name%  
%
population L&5	continent��&<6(EuropeAfrica ,     &�5country��&�<6 (SpainEgypt ,     &�5name��&�<6 (Paris	Barselona ,     &�
5
population��&�<   �p�A   *u�A(   �p�A   *u�A ,     � ,org.apache.spark.version3.0.3 )org.apache.spark.sql.parquet.row.metadata�{"type":"struct","fields":[{"name":"continent","type":"string","null

In [43]:
## Создание DataFrame от локального файла, он лежит рядом с ноутбуком, не на HDFS!!!
import pandas as pd

pdf = pd.read_csv("population.csv")
pdf

,continent,country,name,population
0,Europe,Russia,Moscow,100000000.0
1,NaN,Spain,Madrid,NaN
2,Europe,France,Paris,205000000.0
3,Europe,Germany,Berlin,140000008.0
4,Europe,Spain,Barselona,NaN
5,Africa,Egypt,Cairo,45000001.0
6,Africa,Egypt,Cairo,45000001.0


In [ ]:
test_pdf = (
    spark.createDataFrame(pdf,
                         ) ## создание DataFrame от локально pandas объекта
)

test_pdf

In [85]:
test_pdf.toPandas()

,continent,country,name,population
0,Europe,Russia,Moscow,100000000.000
1,None,Spain,Madrid,NaN
2,Europe,France,Paris,205000000.000
3,Europe,Germany,Berlin,140000008.000
4,Europe,Spain,Barselona,NaN
5,Africa,Egypt,Cairo,45000001.000
6,Africa,Egypt,Cairo,45000001.000


Смотрим на методы взаимодействия с DataFrame:
- select
- filter
- dropDuplicates
- na.fill
- replace
- orderBy

Триггерные действия:
- show
- save
- saveAsTable - просто рассказать

документация со всем, что может пригодиться:
- методы к DataFrame - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html
- методы из sql.functions - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

- Посмотрим .explain() и на сами планы
- alias и когда он нужен
- join и как выглядят их планы вычислений

Будем переодически смотреть на UI и как там все это выглядит

Полезная ссылка - https://habr.com/ru/articles/901078/ - тут написано про Catalyst и как он оптимизирует ваши запросы к данным

In [33]:
## Важно добавить, в sql.functions методах для указания колонок, над которыми данные хотите применить методы
## указывать колонки через перечисление string переменных, либо через метод F.col() - к которому также можно применять свои методы
## Второй подход более предпочтительный, так как к объектам колонок F.col() можно применить дополнительные методы

In [46]:
## Посмотреть как выглядит реализация какой-то задачи на SQL

df.createOrReplaceTempView("test_test") ## создание метки view для обращения к ней через "FROM test_test" 


## Далее всем знакомый вам SQL код
spark.sql("""
    select *
    
    from test_test as a join test_test as b on a.name = b.name
    
    where a.name = 'Moscow'

""").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('a.name = Moscow)
   +- 'Join Inner, ('a.name = 'b.name)
      :- 'SubqueryAlias a
      :  +- 'UnresolvedRelation [test_test]
      +- 'SubqueryAlias b
         +- 'UnresolvedRelation [test_test]

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Project [continent#6, country#7, name#8, population#9L, continent#148, country#149, name#150, population#151L]
+- Filter (name#8 = Moscow)
   +- Join Inner, (name#8 = name#150)
      :- SubqueryAlias a
      :  +- SubqueryAlias test_test
      :     +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
      +- SubqueryAlias b
         +- SubqueryAlias test_test
            +- LogicalRDD [continent#148, country#149, name#150, population#151L], false

== Optimized Logical Plan ==
Join Inner, (name#8 = name#150)
:- Filter (isnotnull(name#8) AND (name#8 = Moscow))
:

In [47]:
## Реализация запроса выше через DataFrame методы

(
    df
    .select("*")
    .join(
        df,
        on=["name"],
        how="inner"
    )
    .filter(F.col("name") == "Moscow")
    .show()
)

## реализовали .select("*") - всех колонок, но так делать не стоит!!! явно указывайте список нужных колонок
## .join() - джоин первого датафрейма ко второму по нужной колонке и с условием inner
## добавление фильтра по колонке name с опр значениями в ней

+------+---------+-------+----------+---------+-------+----------+
|  name|continent|country|population|continent|country|population|
+------+---------+-------+----------+---------+-------+----------+
|Moscow|   Europe| Russia| 100000000|   Europe| Russia| 100000000|
+------+---------+-------+----------+---------+-------+----------+



In [48]:
## разбираем отдельно методы и их планы запросов:
    
(
    df
    .select("name", "continent", "country") ## метод выбора нужных колонок из дата фрейма
    .show()
)

(
    df
    .select("name", "continent", "country")
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала делает скан  коллекции а после вызывает метод Project для выделения нужных колонок из данных
## ПЛАНЫ ЗАПРОСОВ ЧИТАЮТСЯ СНИЗУ ВВЕРХ

## отличия каждого из планов более подробно разобраны на лекции, в блоке про их чтение и определение

+---------+---------+-------+
|     name|continent|country|
+---------+---------+-------+
|   Moscow|   Europe| Russia|
|   Madrid|     null|  Spain|
|    Paris|   Europe| France|
|   Berlin|   Europe|Germany|
|Barselona|   Europe|  Spain|
|    Cairo|   Africa|  Egypt|
|    Cairo|   Africa|  Egypt|
+---------+---------+-------+

== Parsed Logical Plan ==
'Project [unresolvedalias('name, None), unresolvedalias('continent, None), unresolvedalias('country, None)]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string
Project [name#8, continent#6, country#7]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Physical Plan ==
*(1) Project [name#8, continent#6, country#7]
+- *(1) Scan ExistingRDD[continent#6,country#7,name#8,population#9L]



In [49]:
(
    df
    .filter(F.col("name") == "Moscow") ## условие фильтрации данных по значению из колонки, аналог WHERE в SQL
    .show()
)

(
    df
    .filter(F.col("name") == "Moscow")
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала делает скан  коллекции а после применяется метод Filter для проверки того условия,
## что мы указали в методах к DataFrame

+---------+-------+------+----------+
|continent|country|  name|population|
+---------+-------+------+----------+
|   Europe| Russia|Moscow| 100000000|
+---------+-------+------+----------+

== Parsed Logical Plan ==
'Filter ('name = Moscow)
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint
Filter (name#8 = Moscow)
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Filter (isnotnull(name#8) AND (name#8 = Moscow))
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Physical Plan ==
*(1) Filter (isnotnull(name#8) AND (name#8 = Moscow))
+- *(1) Scan ExistingRDD[continent#6,country#7,name#8,population#9L]



In [50]:
(
    df
    .show() ## в таком вызове в данных видим полные дубли по строкам, конкретно в данных по Африке
)

(
    df
    .dropDuplicates() ## метод выявления дублей и их удалние, если аргументов нет - дубли ищутся по всем строкам
    .show()
)

(
    df
    .dropDuplicates() 
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала скан коллекции а после связка трех методов - HashAggregate, Exchange hashpartitioning и снова HashAggregate
## посянение почему так метод реализуется на уровне движка лучше в записи семинара
## я там явно проговариваю логику работы оптимизатора

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| Russia|   Moscow| 100000000|
|     null|  Spain|   Madrid|      null|
|   Europe| France|    Paris| 205000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe|  Spain|Barselona|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
+---------+-------+---------+----------+

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe|Germany|   Berlin| 140000008|
|     null|  Spain|   Madrid|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Europe|  Spain|Barselona|      null|
|   Europe| France|    Paris| 205000000|
|   Europe| Russia|   Moscow| 100000000|
+---------+-------+---------+----------+

== Parsed Logical Plan ==
Deduplicate [continent#6, country#7, name#8, population#9L]
+- LogicalRDD [continent#6, country#7, name#8, popu

In [52]:
(
    df
    .show() 
)

(
    df
    .orderBy(F.asc("continent")) ## метод сортировки данных, обычно полезен для визуализации какой-то части данных в вашем выводе
    .show()
)

(
    df
    .orderBy(F.asc("continent"))
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Важное замечание - тут вызывается Exchange rangepartitioning, который сильно хуже оптимизирован, чем Exchange hashpartitioning
## Метод сортировки данных стоит использовать только для промежуточных вычислений, если в задаче обработки данных у вас
## явно ничего нет про сортировку фрейма данных
## подробнее про виды partitioning стоит прочитать в доке по ссылкам выше

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| Russia|   Moscow| 100000000|
|     null|  Spain|   Madrid|      null|
|   Europe| France|    Paris| 205000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe|  Spain|Barselona|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
+---------+-------+---------+----------+

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|     null|  Spain|   Madrid|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
|   Europe| Russia|   Moscow| 100000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe| France|    Paris| 205000000|
|   Europe|  Spain|Barselona|      null|
+---------+-------+---------+----------+

== Parsed Logical Plan ==
'Sort ['continent ASC NULLS FIRST], true
+- LogicalRDD [continent#6, c

In [53]:
## джойны и их планы запросов
(
    df
    .join(
        df,
        on=["name"],
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор SortMergeJoin подхода в движке
## и предварительные сортировки ключей "Sort [name#8 ASC NULLS FIRST], false, 0" чтобы раскидать их по нодам

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(name))
:- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- LogicalRDD [continent#308, country#309, name#310, population#311L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string, population: bigint, continent: string, country: string, population: bigint
Project [name#8, continent#6, country#7, population#9L, continent#308, country#309, population#311L]
+- Join Inner, (name#8 = name#310)
   :- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- LogicalRDD [continent#308, country#309, name#310, population#311L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7, population#9L, continent#308, country#309, population#311L]
+- Join Inner, (name#8 = name#310)
   :- Filter isnotnull(name#8)
   :  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- Filter isnotnull(name#310)
      +- LogicalRDD [continent#308, country#3

In [54]:
## джойны и их планы запросов
(
    df
    .join(
        F.broadcast(df),
        on=["name"],
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор BroadcastHashJoin, потому что к одному из фреймов явно применили Broadcast метод
## про этот метод есть пояснение в лекции
## также важно: видно что здесь нет этапа сортировки ключей, даже на втором фрейме данных
## отсуствие этого этапа рассчета для джойна и дает нам оптимизацию по времени его работы

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(name))
:- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [continent#319, country#320, name#321, population#322L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string, population: bigint, continent: string, country: string, population: bigint
Project [name#8, continent#6, country#7, population#9L, continent#319, country#320, population#322L]
+- Join Inner, (name#8 = name#321)
   :- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- ResolvedHint (strategy=broadcast)
      +- LogicalRDD [continent#319, country#320, name#321, population#322L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7, population#9L, continent#319, country#320, population#322L]
+- Join Inner, (name#8 = name#321), rightHint=(strategy=broadcast)
   :- Filter isnotnull(name#8)
   :  +- LogicalRDD [continent#6, coun

In [55]:
## джойны и их планы запросов
(
    df.alias("a")
    .join(
        F.broadcast(df.alias("b")),
        on=F.col("a.name") != F.col("b.name"),
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор BroadcastNestedLoopJoin по причине того,
## что условие джйона не вписывается в проверку ключей на равенство + применем Broadcast к одному из фреймов данных
## Сортировки ключей тут также нет
## Но join данных по частям на каждом воркере будет выполняться через вложенный цикл, из-за сложности условия

== Parsed Logical Plan ==
Join Inner, NOT (name#8 = name#332)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- SubqueryAlias b
      +- LogicalRDD [continent#330, country#331, name#332, population#333L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Join Inner, NOT (name#8 = name#332)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- SubqueryAlias b
      +- LogicalRDD [continent#330, country#331, name#332, population#333L], false

== Optimized Logical Plan ==
Join Inner, NOT (name#8 = name#332), rightHint=(strategy=broadcast)
:- Filter isnotnull(name#8)
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- Filter isnotnull(name#332)
   +- LogicalRDD [continent#330, country

In [56]:
## джойны и их планы запросов

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1") ## для след примера выключим автоматический Broadcast
## небольших фреймов данных

(
    df.alias("a")
    .join(
        df.alias("b"),
        on=F.col("a.name") != F.col("b.name"),
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор CartesianProduct, самый "долгий" из физических подходов по вычислению работы джойна
## В силу сложности условия джойна + невозможности применить Broadcast (мы его буквально отключили)
## был выбран CartesianProduct метод
## оптимизаций тут нет, данные распределяются на все возможные паросочетания ключей
## раскидываются на ноды по хэшу и после проверяются по условию на правильно объединения данных

== Parsed Logical Plan ==
Join Inner, NOT (name#8 = name#360)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- SubqueryAlias b
   +- LogicalRDD [continent#358, country#359, name#360, population#361L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Join Inner, NOT (name#8 = name#360)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- SubqueryAlias b
   +- LogicalRDD [continent#358, country#359, name#360, population#361L], false

== Optimized Logical Plan ==
Join Inner, NOT (name#8 = name#360)
:- Filter isnotnull(name#8)
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- Filter isnotnull(name#360)
   +- LogicalRDD [continent#358, country#359, name#360, population#361L], false

== Physical Plan ==
CartesianProduct NOT (name#8 = name#360)
:- *(1) Filter i

In [57]:
## ниже план запроса методов split и explode
## которые превращают строковую колонку в СПИСОК символов
## а после список символов мы раскидываем на уникальные значения для каждой строки, тем самым
## на каждый эл-т из списка создаем новую строку

(
    df
    .withColumn("name_sep", F.split("name", ""))
    .withColumn("name_exp", F.explode("name_sep"))
    .filter(F.col("name_exp") == "s")
    .explain(True)
)

## Логику отработки этого метода можно увидеть на, Physical Plan

== Parsed Logical Plan ==
'Filter ('name_exp = s)
+- Project [continent#6, country#7, name#8, population#9L, name_sep#386, name_exp#393]
   +- Generate explode(name_sep#386), false, [name_exp#393]
      +- Project [continent#6, country#7, name#8, population#9L, split(name#8, , -1) AS name_sep#386]
         +- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, name_sep: array<string>, name_exp: string
Filter (name_exp#393 = s)
+- Project [continent#6, country#7, name#8, population#9L, name_sep#386, name_exp#393]
   +- Generate explode(name_sep#386), false, [name_exp#393]
      +- Project [continent#6, country#7, name#8, population#9L, split(name#8, , -1) AS name_sep#386]
         +- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Filter (isnotnull(name_exp#393) AND (name_exp#393 = s))
+- Generate explode(name_sep#386), false, [

## Сравнение реализаций задачи над RDD и DataFrame

In [59]:
## пример задачи через RDD на подсчет числа упоминаний стран в данных


rdd \
    .map(lambda x: x['country'].lower()) \
    .map( lambda x: (x, 1)) \
    .reduceByKey( lambda x,y: x + y) \
    .collect()


[('france', 1), ('egypt', 2), ('russia', 1), ('spain', 2), ('germany', 1)]

In [60]:
## она же через DataFrame API
(
    df
    .groupBy("country")
    .agg(F.count("*"))
    .explain(True)
)

== Parsed Logical Plan ==
'Aggregate ['country], [unresolvedalias('country, None), count(1) AS count(1)#405L]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
country: string, count(1): bigint
Aggregate [country#7], [country#7, count(1) AS count(1)#405L]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Aggregate [country#7], [country#7, count(1) AS count(1)#405L]
+- Project [country#7]
   +- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Physical Plan ==
*(2) HashAggregate(keys=[country#7], functions=[count(1)], output=[country#7, count(1)#405L])
+- Exchange hashpartitioning(country#7, 200), true, [id=#560]
   +- *(1) HashAggregate(keys=[country#7], functions=[partial_count(1)], output=[country#7, count#409L])
      +- *(1) Project [country#7]
         +- *(1) Scan ExistingRDD[continent#6,country#7,name#8,population#9L]

